# Linear algebra operations in Julia

This is a brief overview of the linear algebra functionalities that are readily available to Julia inside the
[`LinearAlgebra`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/) library. In Julia (as in much of scientific computation), dense linear algebra operations are based on [LAPACK](https://www.netlib.org/lapack/), which is itself built on top of basic linear algebra routines known as [BLAS](https://www.netlib.org/blas/). By default, `LinearAlgebra` ships with `OpenBLAS`, but other, more performant options are also available. See for instance [MKL for Julia](https://github.com/JuliaLinearAlgebra/MKL.jl) (which may or may not perform on Apple silicon processors).

Interfaces for working with both `BLAS` and `LAPACK` routines directly are readily available:

* [BLAS functions](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#BLAS-functions)
* [LAPACK functions](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#man-linalg-lapack-functions)

For working with sparse arrays, one typically uses the [SparseArrays](https://docs.julialang.org/en/v1/stdlib/SparseArrays/) library. We also note a [host of external packages](https://docs.julialang.org/en/v1/stdlib/SparseArrays/#Noteworthy-External-Sparse-Packages), such as the [KrylovKit](https://github.com/Jutho/KrylovKit.jl)
which we also employ in this project.

In [3]:
using LinearAlgebra
using SparseArrays
using BenchmarkTools
using Random

## Some basic notes

First things first, let us briefly discuss basic data containers we would typically use in our numerical calculations - vectors, arrays, matrices (which are actually just an alias for 2D arrays), vectors of vectors ... [This](https://www.matecdev.com/posts/julia-array-initialize.html) is a good starting resource for those wishing to know more.

In [35]:
# to construct a vector: note the square brackets and commas
v1 = [1, 2, 3]
println("Type of v1: ", typeof(v1))
v2 = [1., 2, 3]
println("Type of v2: ", typeof(v2))
# this would also create a vector (of vectors)
v3 = [[1, 2, 3], ["s", "d"], [1.]]
println("Type of v3: ", typeof(v3))
# 

Type of v1: Vector{Int64}
Type of v2: Vector{Float64}
Type of v3: Vector{Vector}


To create an array/ matrix, do the following:

In [37]:
A = [1 2 3; 4 5 6; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [41]:
# matrix is a subtype of the more general Array type
println(Matrix <: Array)
# What about vectors?
println(Vector <: Array)
# what about vectors and matrices?
println(Vector <: Matrix)

true
true
false


When constructing, bear this in mind: 

when constructing vectors, each element can be separated either by commas or semicolons; however, when separating by whitespace, that creates a matrix, which is a different type of entity in Julia.

In [42]:
b1 = [1, 2, 3]
b2 = [1; 2; 3]
b3 = [1 2 3]
println("Type of b1: ", typeof(b1))
println("Type of b2: ", typeof(b2))
println("Type of b3: ", typeof(b3))

Type of b1: Vector{Int64}
Type of b2: Vector{Int64}
Type of b3: Matrix{Int64}


## Array comprehensions, undefined arrays and array preallocation

Of course there are many different ways to construct arrays other than explicit construction, for instance array comprehensions:


In [44]:
# the semicolon at the end suppresses the output
# we are creating a vector of inverse squares of integers from 1 to 1000
vec = [1/i^2 for i in 1:1000];
# let's sum this up
sum(vec)

1.643934566681561

If parentheses are used instead of square brackets, a *generator* object is created. This may sometimes help with the performance, as a generator is not evaluated untill needed. 

In [45]:
genvec = (1/i^2 for i in 1:1000)
sum(genvec)

1.6439345666815615

Let's benchmark the two approaches:

In [66]:
@btime sum([1/i^2 for i in 1:10000])

  6.100 μs (2 allocations: 78.17 KiB)


1.6448340718480614

In [67]:
@btime sum((1/i^2 for i in 1:10000))

  9.666 μs (0 allocations: 0 bytes)


1.6448340718480652

While the former method is faster, it also allocates more. In general, it is advisable to benchmark you code snippets this way to see what yields optimal performance.

For performance reasons, it is typically recommended to initialize arrays of a given type (and size) without any values (not even zeros). To do this, we use keywords such as `Vector{T}, Matrix{T}, Array{T}`, where `T` is a given type, such as `Float64` or `Int64`. The syntax is as follows:

In [73]:
n = 10
A0 = Array{Float64}(undef, n, n);
A1 = Array{Float64, 2}(undef, n, n);
A2 = Matrix{Int64}(undef, n, n);

V0 = Vector{Float64}(undef, n);

The above approach will reserve a certain portion of memory for the containers in advance. As it is not initializing them, this saves some time. But beware, as it may also lead to unwelcome surprises later down the road if you assume the existence of some sensible values (in short: do not assume they were initialized to zero, they may just as well be some gibberish, this can be system and compiler dependent).

To show where this is useful, consider filling a vector with a known number of elements, first using preallocation, then without it. 

In [111]:
n = 100000
@btime begin
   # apparently, using eachindex is also important
   # as it makes a difference compared to 1:n looping
   @inbounds for i in eachindex(V1)
         V1[i] = i
   end
end setup=(V1 = Vector{Int64}(undef, n))

  19.709 μs (0 allocations: 0 bytes)


In [113]:
# now without the preallocation
V2 = Int64[]
@btime begin
    for i in 1:n
        push!(V2, i)
    end
end setup=(V2 = Int64[])

  4.540 ms (298989 allocations: 7.92 MiB)


That appears as a huge difference, both in time and memory. Again, it is important to benchmark your code snippets, as the outcomes are sometimes surprising at a first glance (to be honest, sometimes even after a prolonged stare).

Finally, there is this little trick which we will keep coming back to:

In [120]:
# case 1
n = 100000
# note the dot syntax
@btime V3 .= 1:n setup=(V3 = Vector{Int64}(undef, n));

  23.542 μs (4 allocations: 128 bytes)


We will return back to Julia's [dot syntax for vectorizing functions](https://docs.julialang.org/en/v1/manual/functions/#man-vectorized) shortly. Note that to initialize arrays to zero values, one can use:

In [119]:
# as said before, sometimes the compiler will
# also initialize undef to zeros, but it's best
# not to expect it to avoid headaches.
zeroarr = zeros(Float64, 10, 10)

10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

## Vectorization via the dot syntax

*Any* Julia function can be applied elementwise to any array (or other collection) using the syntax `f.(A)`. Following the official docs:

In [123]:
A = [1. 2. 3.; 4. 5. 6.; 7. 8. 9.]

sin.(A)

3×3 Matrix{Float64}:
  0.841471   0.909297   0.14112
 -0.756802  -0.958924  -0.279415
  0.656987   0.989358   0.412118

Consult the docs for a more thorough explanation, we'll make do with some examples here:

In [130]:
f(x, y) = 3x + 4y;

A = [1., 2., 3.]
B = [4., 5., 6.]

r1 = f.(A, B);
r2 = f.(π, B);


If possible, multiple dotted operations will be *fused* into a single one. In plain terms, this is beneficial, as allocations of intermediate arrays are usually skipped. 

In [137]:
y(x) = x^2
h(x) = cos(x)

y.(h.(A))
# this would achieve the same as
broadcast(x -> y(h(x)), A) == y.(h.(A))

true

Benchmarks below are to prove these methods are comparable/the same in terms of performance. This also goes on to show that quite often, vectorization isn't even needed in Julia, as "for loops are fast" (with some caveats).

In [156]:
@btime y.(h.(X)) setup=(X=collect(1:10000));

  62.500 μs (2 allocations: 78.17 KiB)


In [157]:
@btime broadcast(x -> y(h(x)), X) setup = (X=collect(1:10000));

  62.458 μs (2 allocations: 78.17 KiB)


In [158]:
# compare this to
@btime [y(h(x)) for x in X] setup=(X=collect(1:10000));

  62.542 μs (2 allocations: 78.17 KiB)


In [23]:
f(x) = 3x.^2 + 4x + 7x.^3


fdot(x) = @. 3x^2 + 4x + 7x^3

fdot (generic function with 1 method)

In [25]:
@btime fdot(x);

  469.595 ns (1 allocation: 7.94 KiB)


In [26]:
@btime f(x);

  2.073 μs (6 allocations: 47.62 KiB)


In [27]:
@btime f.(x);

  732.582 ns (3 allocations: 7.97 KiB)
